# ML-Model

## Data Preparation

In [2]:
import pandas as pd

accidents = pd.read_csv('../data/Accidents0514.csv')
vehicles = pd.read_csv('../data/Vehicles0514.csv')

def read_and_join_description(df, col_name):
    col_name_new = col_name.replace('_',' ')
    col_name_new = col_name_new.replace('?','')
    
    col_name_new = col_name_new.replace('Pedestrian Crossing-Human Control','Ped Cross - Human')
    col_name_new = col_name_new.replace('Pedestrian Crossing-Physical Facilities','Ped Cross - Physical')
    col_name_new = col_name_new.replace('Weather Conditions','Weather')
    col_name_new = col_name_new.replace('Road Surface Conditions','Road Surface')
    col_name_new = col_name_new.replace('Urban or Rural Area','Urban Rural')
    col_name_new = col_name_new.replace('Did Police Officer Attend Scene of Accident','Police Officer Attend')
    col_name_new = col_name_new.replace('Pedestrian ','Ped ')
    col_name_new = col_name_new.replace('Bus or Coach Passenger','Bus Passenger')
    col_name_new = col_name_new.replace('Casualty Home Area Type','Home Area Type')
    col_name_new = col_name_new.replace('Vehicle Location-Restricted Lane','Vehicle Location')
    col_name_new = col_name_new.replace('Vehicle Leaving Carriageway','Veh Leaving Carriageway')
    col_name_new = col_name_new.replace('Hit Object off Carriageway','Hit Object Off Carriageway')
    col_name_new = col_name_new.replace('Journey Purpose of Driver','Journey Purpose')
    col_name_new = col_name_new.replace('Age Band of Casualty','Age Band')
    col_name_new = col_name_new.replace('Age Band of Driver','Age Band')
    col_name_new = col_name_new.replace('Propulsion Code','Vehicle Propulsion Code')
    col_name_new = col_name_new.replace('Driver Home Area Type','Home Area Type')
    
    excel_data = pd.read_excel(open('../data/Road-Accident-Safety-Data-Guide.xls', 'rb'),sheet_name=col_name_new) 
    excel_data.columns = excel_data.columns.str.lower()
    excel_data = excel_data.add_prefix(col_name+'_')
    
    final_df = pd.merge(df, excel_data, how = 'left', left_on=col_name, right_on=col_name+'_code')
    final_df.drop([col_name+'_code',col_name], axis=1,inplace=True)
    final_df.columns = final_df.columns.str.replace('_label','')
    return final_df

accidents_full = accidents.copy()

for name in ['Police_Force',
 'Accident_Severity',
 'Day_of_Week',
 'Local_Authority_(District)',
 'Local_Authority_(Highway)',
 '1st_Road_Class',
 'Road_Type',
 'Junction_Detail',
 'Junction_Control',
 '2nd_Road_Class',
 'Pedestrian_Crossing-Human_Control',
 'Pedestrian_Crossing-Physical_Facilities',
 'Light_Conditions',
 'Weather_Conditions',
 'Road_Surface_Conditions',
 'Special_Conditions_at_Site',
 'Carriageway_Hazards',
 'Urban_or_Rural_Area',
 'Did_Police_Officer_Attend_Scene_of_Accident']:
    accidents_full = read_and_join_description(accidents_full, name)
    
vehicles_full = vehicles.copy()

for name in ['Vehicle_Type',
 'Towing_and_Articulation',
 'Vehicle_Manoeuvre',
 'Vehicle_Location-Restricted_Lane',
 'Junction_Location',
 'Skidding_and_Overturning',
 'Hit_Object_in_Carriageway',
 'Vehicle_Leaving_Carriageway',
 'Hit_Object_off_Carriageway',
 '1st_Point_of_Impact',
 'Was_Vehicle_Left_Hand_Drive?',
 'Journey_Purpose_of_Driver',
 'Sex_of_Driver',
 'Age_Band_of_Driver',
 'Propulsion_Code']:
    vehicles_full = read_and_join_description(vehicles_full, name)
    


C:\Users\AlexB\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
vehicles_full.columns

Index(['Accident_Index', 'Vehicle_Reference', 'Age_of_Driver',
       'Engine_Capacity_(CC)', 'Age_of_Vehicle', 'Driver_IMD_Decile',
       'Driver_Home_Area_Type', 'Vehicle_Type', 'Towing_and_Articulation',
       'Vehicle_Manoeuvre', 'Vehicle_Location-Restricted_Lane',
       'Junction_Location', 'Skidding_and_Overturning',
       'Hit_Object_in_Carriageway', 'Vehicle_Leaving_Carriageway',
       'Hit_Object_off_Carriageway', '1st_Point_of_Impact',
       'Was_Vehicle_Left_Hand_Drive?', 'Journey_Purpose_of_Driver',
       'Sex_of_Driver', 'Age_Band_of_Driver', 'Propulsion_Code'],
      dtype='object')

In [9]:
vehicles_full['Vehicle_Type'].drop_duplicates()

0                                            Car
1           Bus or coach (17 or more pass seats)
5                     Motorcycle 125cc and under
14                                 Other vehicle
21                         Motorcycle over 500cc
32                                   Pedal cycle
36           Van / Goods 3.5 tonnes mgw or under
44         Motorcycle over 125cc and up to 500cc
74                         Taxi/Private hire car
91                 Goods 7.5 tonnes mgw and over
132              Goods over 3.5t. and under 7.5t
142                    Motorcycle 50cc and under
255             Minibus (8 - 16 passenger seats)
2072                        Agricultural vehicle
32752                                       Tram
32816                               Ridden horse
52229               Data missing or out of range
2047823                  Motorcycle - unknown cc
2078549                         Mobility scooter
2292246           Goods vehicle - unknown weight
2570110             

In [7]:
accidents_full.columns

Index(['Accident_Index', 'Location_Easting_OSGR', 'Location_Northing_OSGR',
       'Longitude', 'Latitude', 'Number_of_Vehicles', 'Number_of_Casualties',
       'Date', 'Time', '1st_Road_Number', 'Speed_limit', '2nd_Road_Number',
       'LSOA_of_Accident_Location', 'Police_Force', 'Accident_Severity',
       'Day_of_Week', 'Local_Authority_(District)',
       'Local_Authority_(Highway)', '1st_Road_Class', 'Road_Type',
       'Junction_Detail', 'Junction_Control', '2nd_Road_Class',
       'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident'],
      dtype='object')

Used for ML Model: 

Accidents: Day_of_Week, 1st_Road_Class, Weather_Conditions, Road_Surface_Conditions, Special_Conditions_at_Site, Urban_or_Rural_Area, Time of year, Time of day, Longitude, Latitude

Vehicles: Vehicle Types in Accident

To predict: Accident_Severity